In [1]:
%reload_ext autoreload
%autoreload 2

import os
import sys
import numpy as np
from timeit import default_timer as timer
from _collections import OrderedDict
import shutil
from pprint import pprint
start = timer()
HOME = os.path.expanduser("~")
PATH = os.path.join(HOME, 'programming/pipeline_utility')
sys.path.append(PATH)
from utilities.FileLocationManager import FileLocationManager
from utilities.SqlController import SqlController
from utilities.utilities_cvat_neuroglancer import get_structure_number, NumpyToNeuroglancer, get_segment_properties
from utilities.utilities_affine import align_point_sets, align_atlas

Connecting dklab@192.168.1.12:3306


In [2]:
animal = 'MD589'
fileLocationManager = FileLocationManager(animal)
OUTPUT_DIR = os.path.join(fileLocationManager.neuroglancer_data, 'atlas')

In [3]:
sql_controller = SqlController(animal)
thumbnail_dir = os.path.join(fileLocationManager.prep, 'CH1', 'thumbnail_aligned')

print(f'resolution: {sql_controller.scan_run.resolution}')
print(f'width: {sql_controller.scan_run.width}')
print(f'height: {sql_controller.scan_run.height}')
box_w = sql_controller.scan_run.width * sql_controller.scan_run.resolution / 10  # 10 mum scale
box_h = sql_controller.scan_run.height * sql_controller.scan_run.resolution / 10  # 10 mum scale
box_z = len(os.listdir(thumbnail_dir))  # 20 mum scale
box_center = np.array([box_w, box_h, box_z]) / 2

atlasV7_volume = np.zeros((int(box_h), int(box_w), int(box_z)), dtype=np.uint8)
print('Shape of atlas volume', atlasV7_volume.shape)


resolution: 0.452
width: 43700
height: 32400
Shape of atlas volume (1464, 1975, 447)


In [8]:
#src = points moving from DK39
#dst = points moving to DK52
src = 'DK39'
dst = 'MD589'
src_centers = sql_controller.get_centers_dict(src)
#print(src_centers)
dst_centers = sql_controller.get_centers_dict(dst)
#print(dst_centers)
common_keys = src_centers.keys() & dst_centers.keys()
src_centers_dict = [x for x in src_centers if x in common_keys] 
dst_centers_dict = [x for x in dst_centers if x in common_keys] 
print(src)
print(src_centers_dict)
print(dst)
print(dst_centers_dict)
src_centers = [src_centers[x] for x in src_centers if x in common_keys] 
dst_centers = [dst_centers[x] for x in dst_centers if x in common_keys] 
src_centers = np.array(src_centers)
#print('src_centers')
print(src_centers)
dst_centers = np.array(dst_centers)
#print('dst_centers')
print(dst_centers)
R, t = align_point_sets(src_centers.T, dst_centers.T, with_scaling=True)

DK39
['5N_L', '5N_R', 'SC']
MD589
['5N_L', '5N_R', 'SC']
[[37315.4   19924.7     172.5  ]
 [37057.9   18534.5     316.5  ]
 [32793.8   10894.1     233.482]]
[[25863.9 16448.5   160. ]
 [25617.9 16089.    298. ]
 [24976.4 10136.9   220. ]]


In [ ]:
np.shape(src_centers.T)

In [13]:
test_point = np.array([40000,30000, 220])
test_point = test_point.reshape(3,1)
print(test_point.shape)
reference_scales = (0.325, 0.325, 20)
tp = t / reference_scales
R @ test_point 

(3, 1)


array([[-4543.76941802],
       [30179.73326535],
       [11180.81441593]])

In [ ]:
def get_db_structure_infos():
    sqlController = SqlController('MD589')
    db_structures = sqlController.get_structures_dict()
    return db_structures

def get_known_foundation_structure_names():
    known_foundation_structures = ['MVePC', 'DTgP', 'VTA', 'Li', 'Op', 'Sp5C', 'RPC', 'MVeMC', 'APT', 'IPR',
                                   'Cb', 'pc', 'Amb', 'SolIM', 'Pr5VL', 'IPC', '8n', 'MPB', 'Pr5', 'SNR',
                                   'DRD', 'PBG', '10N', 'VTg', 'R', 'IF', 'RR', 'LDTg', '5TT', 'Bar',
                                   'Tz', 'IO', 'Cu', 'SuVe', '12N', '6N', 'PTg', 'Sp5I', 'SNC', 'MnR',
                                   'RtTg', 'Gr', 'ECu', 'DTgC', '4N', 'IPA', '3N', '7N', 'LC', '7n',
                                   'SC', 'LPB', 'EW', 'Pr5DM', 'VCA', '5N', 'Dk', 'DTg', 'LVe', 'SpVe',
                                   'MVe', 'LSO', 'InC', 'IC', 'Sp5O', 'DC', 'Pn', 'LRt', 'RMC', 'PF',
                                   'VCP', 'CnF', 'Sol', 'IPL', 'X', 'AP', 'MiTg', 'DRI', 'RPF', 'VLL']
    return known_foundation_structures

In [ ]:
db_structure_infos = get_db_structure_infos()
known_foundation_structure_names = get_known_foundation_structure_names()
non_db_structure_names = [structure for structure in known_foundation_structure_names if structure not in db_structure_infos.keys()]


In [ ]:
[(number, f'{structure}: {label}') for structure, (label, number) in db_structure_infos.items()]

In [ ]:
segment_properties = [(number, f'{structure}: {label}') for structure, (label, number, sided) in db_structure_infos.items()]
if all_known:
    segment_properties += [(len(db_structure_infos) + index + 1, structure) for index, structure in enumerate(non_db_structure_names)]
